In [2]:
import os
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

conf = (
    SparkConf()
    .setAppName("knmi daily weather to bronze")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", 'minio')
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123')
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
#sqlContext = SQLContext(sc)
spark = SparkSession(sc).builder.getOrCreate()

22/09/30 21:34:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/30 21:34:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.format('delta').load('s3a://bronze-rws/filedata')

In [7]:
df.columns

['NLSitNummer',
 'DatumFileBegin',
 'DatumFileEind',
 'TijdFileBegin',
 'TijdFileEind',
 'FileZwaarte',
 'GemLengte',
 'FileDuur',
 'HectometerKop',
 'HectometerStaart',
 'RouteLet',
 'RouteNum',
 'RouteOms',
 'hectometreringsrichting',
 'KopWegvakVan',
 'KopWegvakNaar',
 'TrajVan',
 'TrajNaar',
 'OorzaakGronddetail',
 'OorzaakVerloop',
 'OorzaakCodeVerloop',
 'OorzaakCode',
 'Oorzaak_1',
 'Oorzaak_2',
 'Oorzaak_3',
 'Oorzaak_4']

In [27]:
from pyspark.sql.functions import col, max, to_date, expr, date_format, regexp_replace
df_silver = df.select(
 col('NLSitNummer'),
 to_date(col('DatumFileBegin'), 'yyyy-MM-dd').alias('DatumFileBegin'),
 to_date(col('DatumFileEind'), 'yyyy-MM-dd').alias('DatumFileEind'),
 date_format(col('TijdFileBegin'),'HH:mm:ss').alias('TijdFileBegin'),
 date_format(col('TijdFileEind'),'HH:mm:ss').alias('TijdFileEind'),
 regexp_replace(col('FileZwaarte'), ',', '.').cast('decimal(17,3)').alias('FileZwaarte'),
 regexp_replace(col('GemLengte'), ',', '.').cast('decimal(17,3)').alias('GemLengte'),
 regexp_replace(col('FileDuur'), ',', '.').cast('decimal(17,3)').alias('FileDuur'),
 col('HectometerKop'),
 col('HectometerStaart'),
 col('RouteLet'),
 col('RouteNum'),
 col('RouteOms'),
 col('hectometreringsrichting'),
 col('KopWegvakVan'),
 col('KopWegvakNaar'),
 col('TrajVan'),
 col('TrajNaar'),
 col('OorzaakGronddetail'),
 col('OorzaakVerloop'),
 col('OorzaakCodeVerloop'),
 col('OorzaakCode'),
 col('Oorzaak_1'),
 col('Oorzaak_2'),
 col('Oorzaak_3'),
 col('Oorzaak_4')
)

In [28]:
df_silver.show()

+-----------+--------------+-------------+-------------+------------+-----------+---------+--------+-------------+----------------+--------+--------+--------+-----------------------+--------------------+---------------+--------------+---------------+--------------------+--------------------+------------------+-----------+--------------------+-------------------+---------+----------------+
|NLSitNummer|DatumFileBegin|DatumFileEind|TijdFileBegin|TijdFileEind|FileZwaarte|GemLengte|FileDuur|HectometerKop|HectometerStaart|RouteLet|RouteNum|RouteOms|hectometreringsrichting|        KopWegvakVan|  KopWegvakNaar|       TrajVan|       TrajNaar|  OorzaakGronddetail|      OorzaakVerloop|OorzaakCodeVerloop|OorzaakCode|           Oorzaak_1|          Oorzaak_2|Oorzaak_3|       Oorzaak_4|
+-----------+--------------+-------------+-------------+------------+-----------+---------+--------+-------------+----------------+--------+--------+--------+-----------------------+--------------------+-------------

In [30]:
df_silver.coalesce(1).write.format("delta").mode("overwrite").option("mergeSchema", "true").save('s3a://silver-rws/filedata')
#df.write.format("csv").mode("overwrite").save('s3a://test-bucket/filedata')